# Chapter 4 - データ前処理 : よりよいトレーニングセットの構築

データの品質と有益な情報の量は, 機械学習アルゴリズムの学習効率を決定するうえで重要.<br />
→ データセットを精査し, 前処理を行うのがよい.

この章では, 以下について勉強する:

- データセットにおける欠陥値の削除と補完
- 機械学習のアルゴリズムに合わせたカテゴリデータの整形
- モデルの構築に適した特徴量の選択